In [5]:
# Импортирование нужных библиотек
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# Чтение данных
X = pd.read_csv('train.csv', index_col = 'PassengerId')
X_test = pd.read_csv('test.csv', index_col = 'PassengerId')

In [6]:
# Удаление строк с пропущенными значениями в целевой переменной
X.dropna(axis=0, subset=['Survived'], inplace=True)
y = X.Survived
X.drop(['Survived'], axis = 1, inplace = True)

# Разделение данных на обучающий и валидационный наборы (на всякий случай, вряд ли это пригодится, так как база маленькая)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Выбор столбцов для обработки
num_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]
cat_cols = ['Sex', 'Embarked']
all_cols = num_cols + cat_cols

In [7]:
X.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
X.dtypes

Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object

In [9]:
X.isnull().sum()

Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [11]:
X_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [12]:
# Создание трансформаторов для двух видов данных
numerical_transformer = SimpleImputer(strategy='mean')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Объединение препроцессора для пайплайна
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ])

# Определение модели машинного обучения
model = RandomForestClassifier(random_state=0)

# Создание пайплайна
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Оценка 'сырой' модели с использованием кросс-валидации
accuracy_scores = cross_val_score(my_pipeline, X[all_cols], y, 
                                  cv=5, 
                                  scoring='accuracy')

print("Accuracy scores:\n", accuracy_scores.mean())

Accuracy scores:
 0.8081225284037412


In [13]:
# Импортирование GridSearchCV для подбора гиперпараметров
from sklearn.model_selection import GridSearchCV

# Определение сетки гиперпараметров для модели RandomForestClassifier
param_grid = {
    'model__n_estimators': [50, 100, 200, 300],
    'model__max_depth': [None, 10, 20, 30],
    'model__min_samples_split': [2, 5, 10]}

# Настройка GridSearchCV для поиска лучших гиперпараметров
grid_search = GridSearchCV(estimator=my_pipeline, param_grid=param_grid, cv=5, scoring='accuracy')

# Поиск лучших гиперпараметров на тренировочных данных
grid_search.fit(X[all_cols], y)

print(f'Лучшие гиперпараметры: {grid_search.best_params_}')
print(f'Лучшая оценка: {grid_search.best_score_}')

Лучшие гиперпараметры: {'model__max_depth': 10, 'model__min_samples_split': 10, 'model__n_estimators': 200}
Лучшая оценка: 0.8350323269097986


In [14]:
# Получение пайплайна с лучшими гиперпараметрами
best_model = grid_search.best_estimator_

# Выполнение предсказаний на тестовом наборе данных
predictions_test = best_model.predict(X_test[all_cols])

In [ ]:
output = pd.DataFrame({'PassengerId': X_test.index,
                       'Survived': predictions_test})
output.to_csv('submission.csv', index=False)